# <center>Dinámica de Langevin</center>

In [1]:
import numpy as np
from plotly import graph_objects as go, figure_factory as ff

np.random.seed(42)

## Clase para la GMM

In [2]:
class GaussianMixture:
    '''Clase para una GMM.'''

    def __init__(self, weights: list, means: list, covariances: list):
        '''
        Parameters:
            - weights: lista de priors de mezcla.
            - means: lista de vectores de media.
            - covariances: lista de matrices de covarianzas.
        '''

        self.dimension = means[0].shape[0]

        assert all(alpha > 0 for alpha in weights) and np.isclose(sum(weights), 1)
        assert all(self.dimension == mu.shape[0] for mu in means)
        assert len(weights) == len(means) == len(covariances)
        for sigma in covariances:
            assert sigma.shape[0] == sigma.shape[1] == self.dimension
            assert np.all(np.linalg.eigvals(sigma) > 0) and np.allclose(sigma, sigma.T)
        
        self.weights = weights
        self.means = means
        self.covariances = covariances


    def density(self, x: float) -> np.ndarray:
        '''
        Calcula la densidad p(x) para la GMM. Se utiliza para el cálculo exacto del score.

        Parameters:
            - x: punto al que se le calculará la densidad.
        
        Returns:
            - density: densidad p(x).
        '''

        density = 0
        for alpha, mu, sigma in zip(self.weights, self.means, self.covariances):
            exponential = np.exp(-1/2 * (x-mu).T @ np.linalg.inv(sigma) @ (x-mu))
            normalization = np.sqrt((2*np.pi)**self.dimension * np.linalg.det(sigma))
            density += alpha * exponential / normalization
        return density


    def score(self, x: float) -> np.ndarray:
        '''
        Calcula el score grad_x(log p(x)) para la GMM. Se utiliza para graficar el
        campo de scores.

        Parameters:
            - x: punto al que se le calculará el score.
        
        Returns:
            - score: score grad_x(log p(x)).
        '''

        density_derivative = 0
        for alpha, mu, sigma in zip(self.weights, self.means, self.covariances):
            exponential = np.exp(-1/2 * (x-mu).T @ np.linalg.inv(sigma) @ (x-mu))
            exp_derivative = exponential * (-np.linalg.inv(sigma) @ (x-mu))
            normalization = np.sqrt((2*np.pi)**self.dimension * np.linalg.det(sigma))
            density_derivative += alpha * exp_derivative / normalization
        
        score = density_derivative / self.density(x)
        return score
    

    def generate_samples(self, n_samples: int) -> np.ndarray:
        '''
        Genera muestras exactas a partir de la GMM.

        Parameters:
            - n_samples: cantidad de muestras.
        
        Returns:
            - samples: muestras generadas.
        '''

        component_indices = np.random.choice(len(self.weights), size=n_samples, p=self.weights)
        samples = []
        for index in component_indices:
            sample = np.random.multivariate_normal(self.means[index], self.covariances[index])
            samples.append(sample)
        return np.array(samples)
    

    @staticmethod
    def grid_evaluation(f: callable, grid_range: list, resolution: int) -> tuple:
            '''
            Evalúa eficientemente una función en una grilla cuadrada (2D) de puntos. Esto
            acelerará considerablemente el cálculo de densidades y score para graficar.

            Parameters:
                - f: función que será evaluada.
                - grid_range: rango de la grilla de la forma [lim_inf, lim_sup].
                - resolution: espaciado de la grilla.
            
            Returns:
                - x: coordenadas x de los puntos de la grilla.
                - y: coordenadas y de los puntos de la grilla.
                - values: valores de la función evaluada en los puntos de la grilla.
            '''

            partition = np.linspace(*grid_range, resolution)
            x, y = map(np.ravel, np.meshgrid(partition, partition))
            values = np.vectorize(lambda x, y: f([x, y]), otypes=[np.ndarray])(x, y)
            return x, y, values
    
    
    def plot_density(self, plot_range: list, density_resolution: int = 100, score_resolution: int = 20) -> go.Figure:
        '''
        Retorna una figura con un mapa de calor de la densidad y un campo vectorial del score de la GMM. No se muestra
        directamente la figura para poder modificarla posteriormente.

        Parameters:
            - plot_range: rango de los ejes. Ambos ejes tendrán el mismo rango.
            - density_resolution: espaciado para visualizar la densidad en el mapa de calor.
            - score_resolution: espaciado para visualizar el score en el campo vectorial.
        
        Returns:
            - fig: figura con la densidad y el score dentro del rango indicado.
        '''

        assert self.dimension == 2, 'La GMM debe ser de dimensión 2.'
        x_score, y_score, scores = self.grid_evaluation(self.score, plot_range, score_resolution)
        x_density, y_density, densities = self.grid_evaluation(self.density, plot_range, density_resolution)
        fig = go.Figure(layout=dict(width=650, height=600, plot_bgcolor='white', margin=dict(l=0, r=0, t=0, b=0), showlegend=False))
        fig.add_traces(ff.create_quiver(x_score, y_score, *zip(*scores), scale=.2).data)
        fig.add_trace(go.Heatmap(z=densities, x=x_density, y=y_density, zsmooth='best', colorscale='Portland'))
        return fig

### GMM en $\mathbb{R}^2$

Se considerará una GMM en $\mathbb{R}^2$ con 3 componentes. La primera componente será una gaussiana de coordenadas independientes (pero no isotrópica), la segunda será una gaussiana con coordenadas correlacionadas, y la tercera componente será una gaussiana isotrópica. Además, las medias estarán lo suficientemente separadas como para que haya tramos de densidad aproximadamente nula entre las modas.

In [3]:
weights = [0.6, 0.3, 0.1]
means = [
    np.array([-7, -7]),
    np.array([ 0,  9]),
    np.array([ 8, -5])
]
covariances = [
    np.array([[1,  0], [ 0, 3]]),
    np.array([[2, -1], [-1, 1]]),
    np.array([[2,  0], [ 0, 2]]),
]

gmm = GaussianMixture(weights, means, covariances)

In [8]:
fig = gmm.plot_density([-20, 20])
means_x, means_y = zip(*gmm.means)
fig.add_trace(go.Scatter(x=means_x, y=means_y, text=gmm.weights, mode='text', textfont=dict(size=16, color='white')))
fig.show()
fig.write_image('images/dm/langevin_gmm.pdf')

## Langevin Monte Carlo

Se implementará el método de Monte Carlo basado en la dinámica de Langevin. La función `langevin_dynamics` generará una única muestra (junto a su trayectoria) para una función de score genérica, mientras que la función `gmm_plot_samples` generará varias muestras de un objeto `GaussianMixture` y graficará sus trayectorias.

In [5]:
def langevin_dynamics(score_function: callable, prior: np.ndarray, epsilon: float = 0.1, steps: int = 100) -> np.ndarray:
    '''
    Genera una muestra a partir de una función de score.

    Parameters:
        - score_function: función para llamar al score.
        - epsilon: espaciado de la discretización (step).
        - steps: cantidad de iteraciones.
        - prior: punto desde el que comenzará la cadena de Markov.

    Returns:
        - trajectory: lista con la trayectoria de la cadena.
    '''

    dimension = prior.shape[0]
    trajectory = [prior]
    for _ in range(steps):
        z = np.random.multivariate_normal(np.zeros(dimension), np.eye(dimension))
        x = trajectory[-1] + epsilon/2 * score_function(trajectory[-1]) + np.sqrt(epsilon) * z
        trajectory.append(x)
    return np.array(trajectory)

### Generación de muestras

In [6]:
def gmm_plot_samples(gmm: GaussianMixture, n_samples: int, prior_range: list) -> None:
    '''
    Muestra la trayectoria de distintas muestras generadas mediante LMC con un prior uniforme.

    Parameters:
        - gmm: mezcla de gaussianas desde donde se obtendrá el score.
        - n_samples: cantidad de muestras que se generarán.
        - prior_range: rango del prior y de los ejes del gráfico.
        - epsilon: epsilon para LMC.
        - steps: steps para LMC.
    '''

    fig = gmm.plot_density(prior_range)
    for _ in range(n_samples):
        prior = np.random.uniform(*prior_range, size=gmm.dimension)
        trajectory = langevin_dynamics(gmm.score, prior)
        fig.add_trace(go.Scatter(x=trajectory[:, 0], y=trajectory[:, 1]))
    fig.show()
    fig.write_image('images/dm/langevin_gmm_samples.pdf')

Se generarán 5 muestras a partir de la GMM definida anteriormente:

In [7]:
gmm_plot_samples(gmm, n_samples=10, prior_range=[-20, 20])

Se observa que las cadenas de Markov definidas con la dinámica de Langevin convergen hacia las modas de la distribución. El problema de este método es que no respeta los priors de mezcla de la GMM. En el gráfico se observa que se generan 3 muestras asociadas a la componente gaussiana menos probable (aquella con prior de mezcla 0.1), mientras que en la componente con prior 0.3 no genera ninguna una muestra. Repitiendo este experimento, la cantidad de muestras generadas tiende a ser la misma para todas las componentes de la mezcla.

Esto ocurrirá siempre que las modas de la distribución desde la que se quieren generar datos estén lo suficientemente separadas y entre ellas hayan valles de densidad aproximadamente nula. En efecto, para dos distribuciones $p_1$ y $p_2$ con soporte disjunto, se puede considerar la mixtura $p_\text{mixture} = \alpha\cdot p_1 + (1-\alpha)\cdot p_2$. Para $x\in \text{supp}(p_1)$, $p_2(x)=0$, por lo que su score vendrá dado por $\nabla_x \log p_\text{mixture} = \nabla_x \log (\alpha\cdot p_1(x)) = \nabla_x (\log\alpha + \log p_1(x)) = \nabla_x \log p_1(x)$. Tomando $\alpha\approx 0$, la componente $p_1$ tendrá un prior de mezcla prácticamente nulo, pero la dinámica de Langevin evolucionará de acuerdo al score de $p_1$ si la inicialización de la cadena de Markov está en su soporte.